In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# 고급 데이터 증강 및 전처리 클래스
class AdvancedTransforms:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomRotation(15),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __call__(self, image):
        image = self.transform(image)
        image_np = np.array(image)
        image_np = self.add_motion_blur(image_np)
        return transforms.ToTensor()(Image.fromarray(image_np))

    def add_motion_blur(self, image_np):
        size = 15
        kernel = np.zeros((size, size))
        kernel[int((size-1)/2), :] = np.ones(size)
        kernel = kernel / size
        image_blurred = cv2.filter2D(image_np, -1, kernel)
        return image_blurred

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = torch.from_numpy(np.array(mask)).long()
        
        return image, mask

# 데이터 경로 및 하이퍼파라미터 설정
image_dir = "path/to/images"
mask_dir = "path/to/masks"
batch_size = 8
num_classes = 25
num_epochs = 20
learning_rate = 0.001

# 데이터셋 및 데이터로더 생성
transform = AdvancedTransforms()
dataset = CustomDataset(image_dir, mask_dir, transform=transform)

# 데이터셋 분할
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 및 전이 학습
model = deeplabv3_mobilenet_v3_large(pretrained=True)  # 사전 훈련된 모델 사용
model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정

# 일부 레이어만 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = False

# 마지막 레이어 학습 가능하도록 설정
for param in model.classifier.parameters():
    param.requires_grad = True

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델 학습
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 모델 검증
model.eval()
val_loss = 0.0
with torch.no_grad():
    for images, masks in val_loader:
        images = images.to(device)
        masks = masks.to(device)
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        val_loss += loss.item()

    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")

# 학습된 모델 저장
torch.save(model.state_dict(), 'finetuned_autonomous_driving_segmentation_model.pth')



In [ ]:
# 모델 로드
def load_model(model_path):
    model = deeplabv3_mobilenet_v3_large(pretrained=False)
    model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

model = load_model('finetuned_autonomous_driving_segmentation_model.pth')


## test

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

# 고급 데이터 증강 및 전처리 클래스
class AdvancedTransforms:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomRotation(15),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __call__(self, image):
        image = self.transform(image)
        # Add motion blur to the image
        image_np = np.array(image)
        image_np = self.add_motion_blur(image_np)
        return transforms.ToTensor()(Image.fromarray(image_np))

    def add_motion_blur(self, image_np):
        size = 15
        kernel = np.zeros((size, size))
        kernel[int((size-1)/2), :] = np.ones(size)
        kernel = kernel / size
        image_blurred = cv2.filter2D(image_np, -1, kernel)
        return image_blurred

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = torch.from_numpy(np.array(mask)).long()
        
        return image, mask

# 데이터 경로 및 하이퍼파라미터 설정
image_dir = "path/to/images"
mask_dir = "path/to/masks"
batch_size = 8
num_classes = 25
num_epochs = 20
learning_rate = 0.001

# 데이터셋 및 데이터로더 생성
transform = AdvancedTransforms()  # 고급 데이터 증강 적용

dataset = CustomDataset(image_dir, mask_dir, transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 및 저장된 가중치 불러오기
model = deeplabv3_mobilenet_v3_large(pretrained=False)
model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정

# 학습된 모델이 저장된 파일을 로드
model.load_state_dict(torch.load('best_autonomous_driving_segmentation_model.pth'), strict=False)

# 일부 레이어만 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = False

# 필요한 레이어만 학습 가능하도록 설정 (예: 마지막 레이어)
for param in model.classifier.parameters():
    param.requires_grad = True

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# 모델 학습
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 모델 검증
model.eval()
val_loss = 0.0
with torch.no_grad():
    for images, masks in val_loader:
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        val_loss += loss.item()
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")

# 학습된 모델 저장
torch.save(model.state_dict(), f'finetuned_autonomous_driving_segmentation_model{3}.pth')


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/images'